<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

<img src = "https://github.com/Jangrae/img/blob/master/mobile2.png?raw=true" width=800 align="left"/>

# 실습 내용

- Mobile 데이터로 모델링합니다.
- Logistic Regression 알고리즘으로 모델링합니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [2]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format='retina'

In [3]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/mobile_cust_churn.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [4]:
# 상위 몇 개 행 확인
data.head()

,id,COLLEGE,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,REPORTED_SATISFACTION,REPORTED_USAGE_LEVEL,CONSIDERING_CHANGE_OF_PLAN,CHURN
0,1,0,31953,0,6,313378,161,0,4,unsat,little,no,STAY
1,2,1,36147,0,13,800586,244,0,6,unsat,little,considering,STAY
2,3,1,27273,230,0,305049,201,16,15,unsat,very_little,perhaps,STAY
3,4,0,120070,38,33,788235,780,3,2,unsat,very_high,considering,LEAVE
4,5,1,29215,208,85,224784,241,21,1,very_unsat,little,never_thought,STAY


**데이터 설명**

- COLLEGE: 대학 졸업여부
- INCOME: 연수입
- OVERAGE: 월평균 초과사용 시간(분)
- LEFTOVER: 월평균 잔여시간비율(%)
- HOUSE: 집값
- HANDSET_PRICE: 스마트폰 가격
- OVER_15MINS_CALLS_PER_MONTH: 월평균 장기통화(15분이상) 횟수
- AVERAGE_CALL_DURATION: 평균 통화 시간
- REPORTED_SATISFACTION: 만족도 설문조사 결과
- REPORTED_USAGE_LEVEL: 사용도 자가진단 결과
- CONSIDERING_CHANGE_OF_PLAN: 향후 변경계획 설문조사 결과
- CHURN: 이탈(번호이동) 여부 (Target 변수)

In [5]:
# 변수 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   id                           20000 non-null  int64 
 1   COLLEGE                      20000 non-null  int64 
 2   INCOME                       20000 non-null  int64 
 3   OVERAGE                      20000 non-null  int64 
 4   LEFTOVER                     20000 non-null  int64 
 5   HOUSE                        20000 non-null  int64 
 6   HANDSET_PRICE                20000 non-null  int64 
 7   OVER_15MINS_CALLS_PER_MONTH  20000 non-null  int64 
 8   AVERAGE_CALL_DURATION        20000 non-null  int64 
 9   REPORTED_SATISFACTION        20000 non-null  object
 10  REPORTED_USAGE_LEVEL         20000 non-null  object
 11  CONSIDERING_CHANGE_OF_PLAN   20000 non-null  object
 12  CHURN                        20000 non-null  object
dtypes: int64(9), object(4)
memory u

In [7]:
# 기술통계 확인
data.describe()

,id,COLLEGE,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,0.502400,80281.447750,85.979550,23.898650,493155.264250,389.615100,8.000700,6.002250
std,5773.647028,0.500007,41680.586319,85.992324,26.816645,252407.884692,213.820682,8.925418,4.402558
min,1.000000,0.000000,20007.000000,-2.000000,0.000000,150002.000000,130.000000,0.000000,1.000000
25%,5000.750000,0.000000,42217.000000,0.000000,0.000000,263714.250000,219.000000,1.000000,2.000000
50%,10000.500000,1.000000,75366.500000,59.000000,14.000000,452259.500000,326.000000,4.000000,5.000000
75%,15000.250000,1.000000,115881.750000,179.000000,41.000000,702378.000000,533.250000,15.000000,10.000000
max,20000.000000,1.000000,159983.000000,335.000000,89.000000,999996.000000,899.000000,29.000000,15.000000


In [8]:
# target 값 개수 확인
data['CHURN'].value_counts()

CHURN
STAY     10148
LEAVE     9852
Name: count, dtype: int64

In [11]:
# 상관관계 확인
dir(data.corr(numeric_only=True))

['AVERAGE_CALL_DURATION',
 'COLLEGE',
 'HANDSET_PRICE',
 'HOUSE',
 'INCOME',
 'LEFTOVER',
 'OVERAGE',
 'OVER_15MINS_CALLS_PER_MONTH',
 'T',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__dataframe__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__non

# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) 변수 제거**

In [12]:
# 제거 대상: id
drop_cols = ['id']

# 변수 제거
data = data.drop(drop_cols, axis=1)

# 확인
data.head(1)

,COLLEGE,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,REPORTED_SATISFACTION,REPORTED_USAGE_LEVEL,CONSIDERING_CHANGE_OF_PLAN,CHURN
0,0,31953,0,6,313378,161,0,4,unsat,little,no,STAY


**2) x, y 분리**

In [13]:
# Target 설정
target = 'CHURN'

# 데이터 분리
X = data.drop(target, axis=1)
y = data.loc[:,target]

**3) 가변수화**

In [15]:
# 가변수화 대상: REPORTED_SATISFACTION, REPORTED_USAGE_LEVEL, CONSIDERING_CHANGE_OF_PLAN
dumm_cols = ['REPORTED_SATISFACTION', 'REPORTED_USAGE_LEVEL', 'CONSIDERING_CHANGE_OF_PLAN']

# 가변수화
X = pd.get_dummies(X, dumm_cols, drop_first = True, dtype =int)

# 확인
X.columns

Index(['COLLEGE', 'INCOME', 'OVERAGE', 'LEFTOVER', 'HOUSE', 'HANDSET_PRICE',
       'OVER_15MINS_CALLS_PER_MONTH', 'AVERAGE_CALL_DURATION',
       'REPORTED_SATISFACTION_sat', 'REPORTED_SATISFACTION_unsat',
       'REPORTED_SATISFACTION_very_sat', 'REPORTED_SATISFACTION_very_unsat',
       'REPORTED_USAGE_LEVEL_high', 'REPORTED_USAGE_LEVEL_little',
       'REPORTED_USAGE_LEVEL_very_high', 'REPORTED_USAGE_LEVEL_very_little',
       'CONSIDERING_CHANGE_OF_PLAN_considering',
       'CONSIDERING_CHANGE_OF_PLAN_never_thought',
       'CONSIDERING_CHANGE_OF_PLAN_no', 'CONSIDERING_CHANGE_OF_PLAN_perhaps'],
      dtype='object')

**4) 학습용, 평가용 데이터 분리**

In [16]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 7:3으로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# 4.모델링

- 본격적으로 모델을 선언하고 학습하고 평가하는 과정을 진행합니다.

In [17]:
# 1단계: 불러오기
# 이탈 여부, 분류, 로지스틱 회귀, confusion matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report


In [18]:
# 2단계: 선언하기
model = LogisticRegression()

In [19]:
# 3단계: 학습하기
model.fit(X_train, y_train)

LogisticRegression()

In [20]:
# 4단계: 예측하기
y_pred = model.predict(X_test)

In [40]:
# 5단계: 평가하기
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[1834 1175]
 [1059 1932]]
              precision    recall  f1-score   support

       LEAVE       0.63      0.61      0.62      3009
        STAY       0.62      0.65      0.63      2991

    accuracy                           0.63      6000
   macro avg       0.63      0.63      0.63      6000
weighted avg       0.63      0.63      0.63      6000



In [26]:
print(y_test.values[:20]) # 실제값
print(y_pred[:20]) # 예측값

['LEAVE' 'STAY' 'LEAVE' 'LEAVE' 'STAY' 'LEAVE' 'STAY' 'STAY' 'STAY'
 'LEAVE' 'LEAVE' 'LEAVE' 'STAY' 'LEAVE' 'LEAVE' 'STAY' 'STAY' 'STAY'
 'STAY' 'STAY']
['STAY' 'LEAVE' 'LEAVE' 'LEAVE' 'LEAVE' 'STAY' 'STAY' 'STAY' 'STAY'
 'LEAVE' 'STAY' 'STAY' 'LEAVE' 'LEAVE' 'STAY' 'LEAVE' 'LEAVE' 'STAY'
 'STAY' 'STAY']


In [32]:
# 확률 확인
p = model.predict_proba(X_test) # 여러 변수들을 준 결과 LEAVE와 STAY 순서로(사전순) 확률이 주어진다.
p # LEAVE, STAY로 정렬 되어 있다.

array([[0.40077428, 0.59922572],
       [0.65348502, 0.34651498],
       [0.74522822, 0.25477178],
       ...,
       [0.57838097, 0.42161903],
       [0.45987557, 0.54012443],
       [0.66359245, 0.33640755]])

In [45]:
p1 = p[:,1] # STAY
p1[:20]

array([0.59922572, 0.34651498, 0.25477178, 0.19602993, 0.49401176,
       0.69041093, 0.5022684 , 0.79446   , 0.53543994, 0.29637205,
       0.5364713 , 0.64347447, 0.45068209, 0.30018108, 0.68469134,
       0.48878371, 0.34282399, 0.58992367, 0.55776038, 0.60670276])

In [49]:
# 임계값 조정해보기
arr = ['STAY' if i >= 0.45 else 'LEAVE' for i in p1]
y_pred2 = np.array(arr)
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

       LEAVE       0.68      0.51      0.58      3009
        STAY       0.61      0.76      0.68      2991

    accuracy                           0.64      6000
   macro avg       0.65      0.64      0.63      6000
weighted avg       0.65      0.64      0.63      6000



In [47]:
print(y_pred[:20])
print(y_pred2[:20])

['STAY' 'LEAVE' 'LEAVE' 'LEAVE' 'LEAVE' 'STAY' 'STAY' 'STAY' 'STAY'
 'LEAVE' 'STAY' 'STAY' 'LEAVE' 'LEAVE' 'STAY' 'LEAVE' 'LEAVE' 'STAY'
 'STAY' 'STAY']
['STAY' 'LEAVE' 'LEAVE' 'LEAVE' 'LEAVE' 'STAY' 'STAY' 'STAY' 'STAY'
 'LEAVE' 'STAY' 'STAY' 'LEAVE' 'LEAVE' 'STAY' 'LEAVE' 'LEAVE' 'STAY'
 'STAY' 'STAY']
